In [2]:
import torch

In [3]:
def mean_with_min_neighbour(x):
    out = []
    for i in range(len(x)):
        if i-1 < 0:
            out.append(torch.mean(x[i:i+2], dim=0))
        elif i+1 < len(x):
            frame = torch.stack([x[i-1], x[i+1]])
            diffs = torch.stack([abs(x[i]-x[i-1]), abs(x[i]-x[i+1])])
            index = torch.min(diffs, dim=0).indices
            minval = torch.stack([j[idx] for j, idx in zip(frame.T, index)])
            out.append(torch.mean(torch.stack([minval, x[i]]), dim=0))
        else:
            out.append(torch.mean(x[i-1:i+1], dim=0))
            
    out = torch.stack(out, dim=0)
    
    return out

test = torch.tensor([[1,5,3],[7,4,6],[9,2,8]],dtype=torch.float32)
test = test.to("cuda")

meaned_v = mean_with_min_neighbour(test).type(torch.int8)
meaned_h = mean_with_min_neighbour(torch.tranpose(test, 0, 1)).type(torch.int8)

print(test)
print(meaned_v)
print(meaned_h)

# Calculate difference of the means to the original values
diff_v = test-meaned_v
diff_h = test-meaned_h

print(diff_v)
print(diff_h)

diff2cal = torch.where(abs(diff_h) > abs(diff_v), diff_v, diff_h)
print(test - diff2cal)

AttributeError: module 'torch' has no attribute 'tranpose'